![Alt text](http://www.gannett-cdn.com/-mm-/1583e6ea8155b5d684fb3fa9ebb795b250e91f9a/c=0-113-909-796&r=x408&c=540x405/local/-/media/2016/10/20/USATODAY/USATODAY/636125763782842649-MetLife-Logo-Reduced-copy.jpg)

# Metlife Sample Notebook
This is an attempt to mimic much of the work the Metlife team did with data cleansing

## Read a csv
The Metlife team worked with many csv files

In [1]:
!wget https://raw.githubusercontent.com/JosephKambourakisIBM/Meetup/master/SAheart.csv


--2017-02-14 12:20:15--  https://raw.githubusercontent.com/JosephKambourakisIBM/Meetup/master/SAheart.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21499 (21K) [text/plain]
Saving to: ‘SAheart.csv.1’

100%[======================================>] 21,499      --.-K/s   in 0.001s  

2017-02-14 12:20:15 (34.0 MB/s) - ‘SAheart.csv.1’ saved [21499/21499]



In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

rawdata = spark.read.option('header', True).csv("SAheart.csv")

## Look at the first five rows

In [3]:
rawdata.take(5)

[Row(sbp=u'160', tobacco=u'12', ldl=u'5.73', adiposity=u'23.11', famhist=u'Present', typea=u'49', obesity=u'25.3', alcohol=u'97.2', age=u'52', chd=u'1'),
 Row(sbp=u'144', tobacco=u'0.01', ldl=u'4.41', adiposity=u'28.61', famhist=u'Absent', typea=u'55', obesity=u'28.87', alcohol=u'2.06', age=u'63', chd=u'1'),
 Row(sbp=u'118', tobacco=u'0.08', ldl=u'3.48', adiposity=u'32.28', famhist=u'Present', typea=u'52', obesity=u'29.14', alcohol=u'3.81', age=u'46', chd=u'0'),
 Row(sbp=u'170', tobacco=u'7.5', ldl=u'6.41', adiposity=u'38.03', famhist=u'Present', typea=u'51', obesity=u'31.99', alcohol=u'24.26', age=u'58', chd=u'1'),
 Row(sbp=u'134', tobacco=u'13.6', ldl=u'3.5', adiposity=u'27.78', famhist=u'Present', typea=u'60', obesity=u'25.99', alcohol=u'57.34', age=u'49', chd=u'1')]

## Clean the data

In [4]:
cleaneddata = rawdata.withColumn("sbp", rawdata["sbp"].cast('float'))\
.withColumn("tobacco", rawdata["tobacco"].cast('float'))\
.withColumn("ldl", rawdata["ldl"].cast('float'))\
.withColumn("adiposity", rawdata["adiposity"].cast('float'))\
.withColumn("typea", rawdata["typea"].cast('int'))\
.withColumn("obesity", rawdata["obesity"].cast('float'))\
.withColumn("alcohol", rawdata["alcohol"].cast('float'))\
.withColumn("age", rawdata["age"].cast('int'))\
.withColumn("chd", rawdata["chd"].cast('float')) #Our model requires the dependent variable to be a float
cleaneddata.printSchema()

root
 |-- sbp: float (nullable = true)
 |-- tobacco: float (nullable = true)
 |-- ldl: float (nullable = true)
 |-- adiposity: float (nullable = true)
 |-- famhist: string (nullable = true)
 |-- typea: integer (nullable = true)
 |-- obesity: float (nullable = true)
 |-- alcohol: float (nullable = true)
 |-- age: integer (nullable = true)
 |-- chd: float (nullable = true)



## Perform transformations

In [5]:
cleaneddata1 = cleaneddata.withColumn("sbp", cleaneddata["sbp"]/10)
cleaneddata1.first()

Row(sbp=16.0, tobacco=12.0, ldl=5.730000019073486, adiposity=23.110000610351562, famhist=u'Present', typea=49, obesity=25.299999237060547, alcohol=97.19999694824219, age=52, chd=1.0)

In [6]:
cleaneddata1 = cleaneddata.withColumn("sbpldl", cleaneddata["sbp"]*cleaneddata['ldl'])
cleaneddata1.first()

Row(sbp=160.0, tobacco=12.0, ldl=5.730000019073486, adiposity=23.110000610351562, famhist=u'Present', typea=49, obesity=25.299999237060547, alcohol=97.19999694824219, age=52, chd=1.0, sbpldl=916.7999877929688)

In [7]:
def roundT(x):
    if x > 6:
        return x
    else:
        return 0.0

In [8]:
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType, FloatType
roundT = UserDefinedFunction(roundT,FloatType())
df = cleaneddata.withColumn('tobacco', roundT(cleaneddata['tobacco']))
df.take(6)

[Row(sbp=160.0, tobacco=12.0, ldl=5.730000019073486, adiposity=23.110000610351562, famhist=u'Present', typea=49, obesity=25.299999237060547, alcohol=97.19999694824219, age=52, chd=1.0),
 Row(sbp=144.0, tobacco=0.0, ldl=4.409999847412109, adiposity=28.610000610351562, famhist=u'Absent', typea=55, obesity=28.8700008392334, alcohol=2.059999942779541, age=63, chd=1.0),
 Row(sbp=118.0, tobacco=0.0, ldl=3.4800000190734863, adiposity=32.279998779296875, famhist=u'Present', typea=52, obesity=29.139999389648438, alcohol=3.809999942779541, age=46, chd=0.0),
 Row(sbp=170.0, tobacco=7.5, ldl=6.409999847412109, adiposity=38.029998779296875, famhist=u'Present', typea=51, obesity=31.989999771118164, alcohol=24.260000228881836, age=58, chd=1.0),
 Row(sbp=134.0, tobacco=13.600000381469727, ldl=3.5, adiposity=27.780000686645508, famhist=u'Present', typea=60, obesity=25.989999771118164, alcohol=57.34000015258789, age=49, chd=1.0),
 Row(sbp=132.0, tobacco=6.199999809265137, ldl=6.46999979019165, adiposity